In [1]:
from os import getcwd, chdir
from os.path import expanduser, join
from glob import glob
prj_root = expanduser("~/Dropbox/ABACUS/Workspace/LeapArticulatorQt")
chdir(prj_root)
prefix = "logs/logs/orange_squares"
files = glob(join(prj_root, prefix,'*.*.exp.log'))
# prefix = "../logs/logs/orange_squares"
# files = files[:2]
# files = map(lambda x: "logs/discrete/%s.exp.log" % x, "D13200321014.2".split(","))
files

['/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS1.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS13.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS16.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS18.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS19.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS2.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS23.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS24.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS21.1.exp.log',
 '/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS3.1.exp.log',
 '/home/kerem

In [2]:
%%writefile ProfileStreamlineNugget.py
import sys
def do_it(file_id=sys.argv[1], units=sys.argv[2], parallel=sys.argv[3], skip_phases=sys.argv[4:-1],
         prefix=sys.argv[-1]):
    from leaparticulator.notebooks.StreamlinedDataAnalysisGhmm import analyze_log_file_in_phases_by_condition
    from leaparticulator import constants
    try:
        print "skip_phase=%s" % skip_phases
        print "parallel? %s" % parallel
        print "prefix=%s" % prefix
        analyze_log_file_in_phases_by_condition(file_id, nstates=range(2,26), trials=100, iter=1000, 
                                                parallel=parallel, units=units,
                                            skip_phases=skip_phases, prefix=prefix)
    except Exception, err:
        print err
if __name__ == "__main__":
    do_it()

Overwriting ProfileStreamlineNugget.py


In [3]:
%load_ext autoreload
%autoreload 2
import warnings, sys
warnings.filterwarnings('ignore')
from leaparticulator.notebooks.StreamlinedDataAnalysisGhmm import analyze_log_file_in_phases_by_condition
from leaparticulator import constants #, GHmmWrapper, gc
from itertools import product
from subprocess import Popen, PIPE, os, STDOUT
from ProfileStreamlineNugget import do_it
files = files[:5]
files_n_units = product(files, [constants.XY, constants.AMP_AND_FREQ, constants.AMP_AND_MEL], range(3))
dd = os.getcwd()
error = []
warning = []
output = []
# skip = [11,14, 17]
# skip = lrange(115,226)
skip = []
# 11 is a problem, again

# skip = range(207)
# return
p = None
from datetime import datetime
log_file = open("logs/StreamlineLog-OrangeSquares.%s.log" % datetime.now(), 'w', 0)
for i, (f, unit, phase) in enumerate(files_n_units):
#         if i <= 13:
#             continue
        log_file.write("**********************************************")
        print "**********************************************"
        log_file.write(f)
        print f
        ff = f.split("/")[-1][:-8]
        cond = ff.split(".")[-1]
        if cond == "master":
            cond = "1"
        multivariate = ("1" in cond and phase==2) or ("2" in cond and phase==1)
#         if i not in [11,14,17]:
#             continue
        if i in skip: #or not multivariate:
            continue
        print "Multivariate?", multivariate
#         if i in skip or i % 3 == 2:#< 11:# or i > 5:
#             # 3 (11) was a problem
#             continue
        status = "Doing file/unit/phase combination no.#%d/%d: %s, phase%s (unit: %s) (cond: %s)" % (i+1, len(files)*3*3, ff, phase, unit, cond)
        print status
        log_file.write(status)
#         do_it(ff, unit)
#         newcode = code % (ff, unit)
        skip_phase = map(str, list(set(range(3)) - set([phase])))
        do_it(ff,unit,False,skip_phase, prefix)
#         p = Popen(('python ProfileStreamlineNugget.py %s %s %s %s %s' % (ff, 
#                                                                       unit,
#                                                                       True,
#                                                                       " ".join(skip_phase),  
#                                                                       prefix)).split(),
#                                                                       stdout=PIPE, 
#                                                                       stderr=STDOUT,
#                                                                       cwd=dd)
# #         do_it(ff, unit, True, skip_phase, prefix=prefix)
#         line = " "
#         while line:
# #             line = p.stdout.readline()
# #             if not line:
# #             output.append(line)
# #             print line.rstrip()
#             if "GHMM" not in str(line):
#                 print line.rstrip()
#             log_file.write(line)
            
# #             else:
# #                 warning.append(line)
#             line = p.stdout.readline()
            
# #             line += "\n" + p.stderr.readline()
#             # this prevents multivariate models from failing
#             # when in cluster mode for whatever reason
# #             if p.stderr:
# #                 a = "%s" % p.stderr.readlines()
# #                 del a
#             sys.stdout.flush()
#         print "Return code:", p.returncode
#         error = p.stderr.readlines()#communicate()[1]
#         print "OUT\n", std
#         print "ERR\n", err
#         gc.collect()

/shared/Dropbox/ABACUS/Workspace/LeapArticulatorQt
**********************************************
/home/kerem/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS1.1.exp.log
Multivariate? False
Doing file/unit/phase combination no.#1/45: OS1.1, phase0 (unit: xy) (cond: 1)
skip_phase=['1', '2']
parallel? False
prefix=logs/logs/orange_squares
Starting phase by phase analysis, controlled for conditions (units: xy)... 
Working dir: /shared/Dropbox/ABACUS/Workspace/LeapArticulatorQt 
Condition 1 
Doing phase#0 
Loading log file: /shared/Dropbox/ABACUS/Workspace/LeapArticulatorQt/logs/logs/orange_squares/OS1.1.exp.log... 
Loaded. 
Sample data: [-65.35152435302734, -65.3702392578125, -65.8470458984375] 
Doing 2 state models... 


KeyboardInterrupt: 